In [106]:
#!/usr/bin/env python3
from __future__ import annotations
import socket, random, re, sys, time
from math import isqrt

try:
    import gmpy2
    from gmpy2 import mpz, gcd, is_prime as g_isprime
except ImportError:
    import subprocess, sys as _s
    subprocess.check_call([_s.executable, "-m", "pip", "install", "gmpy2"])
    import gmpy2
    from gmpy2 import mpz, gcd, is_prime as g_isprime

# optional sympy
try:
    from sympy.ntheory import factorint as sym_factorint
except ImportError:
    sym_factorint = None

# pre‑compute primes ≤ 80 k
SMALL_PRIMES = []
for i in range(2, 80_000):
    for p in SMALL_PRIMES:
        if p * p > i:
            break
        if i % p == 0:
            break
    else:
        SMALL_PRIMES.append(i)

# Pollard‑Brent

def pollard_brent(n: int, it_max: int, seed: int) -> int | None:
    """Return a non‑trivial factor of *n* or None within *it_max* iterations.
    Safe‑guard: if n ≤ 3, return None immediately (avoids randrange empty range)."""

    if n <= 3:
        return None  # 1, 2, 3 are prime/small → nothing to do
    if n & 1 == 0:
        return 2
    rnd = random.Random(seed)
    n_mpz = mpz(n)
    y = mpz(rnd.randrange(2, n - 1))
    c = mpz(rnd.randrange(1, n - 1))
    m, g, r, q, it = 256, mpz(1), 1, mpz(1), 0
    while g == 1:
        x = y
        for _ in range(r):
            y = (y * y + c) % n_mpz
            it += 1
            if it >= it_max:
                return None
        k = 0
        while k < r and g == 1:
            ys = y
            lim = min(m, r - k)
            for _ in range(lim):
                y = (y * y + c) % n_mpz
                q = (q * abs(x - y)) % n_mpz
                it += 1
                if it >= it_max:
                    return None
            g = gcd(q, n_mpz)
            k += m
        r <<= 1
    if g == n_mpz:
        while True:
            ys = (ys * ys + c) % n_mpz
            g = gcd(abs(x - ys), n_mpz)
            if g > 1:
                break
    return int(g)

# divisors generator

def divisors(factors: list[int]):
    d = {1}
    for p in factors:
        d |= {x * p for x in d}
    return d

# attempt triplet

def attempt(S1: int, S3: int, facs: list[int]):
    P = (S1 ** 3 - S3) // 3
    T = 2 * S1
    for p in divisors(facs):
        if P % p:
            continue
        qr = P // p
        t = T - p
        Δ = t * t - 4 * qr
        if Δ < 0:
            continue
        s = isqrt(Δ)
        if s * s != Δ:
            continue
        q = (t + s) // 2
        r = t - q
        x = (p + r - q) // 2
        y = (p + q - r) // 2
        z = S1 - x - y
        if min(x, y, z) > 0 and x ** 3 + y ** 3 + z ** 3 == S3:
            return tuple(sorted((x, y, z)))
    return None

# solve round

def solve_round(S1: int, S3: int):
    P = (S1 ** 3 - S3) // 3
    composites = [P]
    facs: list[int] = []
    # strip small primes
    for p in SMALL_PRIMES:
        i = 0
        while i < len(composites):
            n = composites[i]
            if n % p == 0:
                while n % p == 0:
                    n //= p
                    facs.append(p)
                composites[i] = n
            if composites[i] == 1:
                composites.pop(i)
            else:
                i += 1
        if not composites:
            break
    res = attempt(S1, S3, facs + composites)
    if res:
        return res

    start = time.perf_counter()
    BUDGET = 4.9
    # factor opportunistically
    while composites and (time.perf_counter() - start) < BUDGET:
        n = composites.pop()
        if g_isprime(n):
            facs.append(n)
            if (res := attempt(S1, S3, facs + composites)):
                return res
            continue
        # Pollard loop
        it = 100_000
        found = None
        while it <= 6_400_000 and (time.perf_counter() - start) < BUDGET - 0.4 and not found:
            found = pollard_brent(n, it, random.randrange(1 << 62))
            it <<= 1
        if not found and sym_factorint and (time.perf_counter() - start) < BUDGET - 0.3:
            try:
                d = next(iter(sym_factorint(n, multiple=True)))
                found = d if d not in (1, n) else None
            except Exception:
                pass
        if not found:
            facs.append(n)  # treat as prime-like; still may succeed
        else:
            a, b = found, n // found
            composites.extend([a, b])
        if (res := attempt(S1, S3, facs + composites)):
            return res
    raise RuntimeError("Triplet non trouvé – round >5 s")

# socket wrapper
class Sock:
    def __init__(self, host, port):
        self.s = socket.create_connection((host, port))
        self.buf = bytearray()
    def _fill(self):
        chunk = self.s.recv(4096)
        if not chunk:
            raise EOFError
        self.buf += chunk
    def recv_until(self, token: bytes):
        while token not in self.buf:
            self._fill()
        idx = self.buf.find(token) + len(token)
        data, self.buf = self.buf[:idx], self.buf[idx:]
        return bytes(data)
    def recv_line(self):
        return self.recv_until(b"\n")
    def send_line(self, txt: str | bytes):
        if isinstance(txt, str):
            txt = txt.encode()
        self.s.sendall(txt + b"\n")
    def close(self):
        self.s.close()

# main

def main():
    io = Sock("challenges.404ctf.fr", 30369)
    io.recv_until(b"?")
    io.send_line("botgalactique")
    for i in range(1, 101):
        S1 = int(re.search(r"= (\d+)", io.recv_line().decode()).group(1))
        S3 = int(re.search(r"= (\d+)", io.recv_line().decode()).group(1))
        t0 = time.perf_counter()
        x, y, z = solve_round(S1, S3)
        print(f"Rnd {i:3d}: {1e3*(time.perf_counter()-t0):7.2f} ms → {x},{y},{z}")
        io.recv_until(b"?")
        io.send_line(f"{x},{y},{z}")
    txt = io.recv_until(b"}").decode(errors="replace")
    io.close()
    m = re.search(r"404CTF\{[^}]+\}", txt)
    print("\n🏁 FLAG:" if m else txt, m.group(0) if m else "")

if __name__ == "__main__":
    try:
        main()
    except KeyboardInterrupt:
        sys.exit()


Rnd   1: 2730.61 ms → 12655587245401762769,12704850264010513079,13674845100921713153
Rnd   2:   34.30 ms → 10101021946092541099,11524654953515696627,18165350783947371313
Rnd   3:   27.82 ms → 10048979946622494509,13969175026727463539,15108086414393389927
Rnd   4:  330.17 ms → 10033398391071063703,10108041314682600329,16052572334414048737
Rnd   5:  839.30 ms → 11775442476096902819,12008813661785062783,14343298930433777441
Rnd   6:   18.86 ms → 9396737163418926929,12119364370977600551,12277397565192140621
Rnd   7: 2897.88 ms → 15967416231401797933,16819480282696765381,18005336392228373311
Rnd   8:  445.64 ms → 10328120798602215943,12329452155079450747,16213573435284599081
Rnd   9:   35.03 ms → 9682401032986793233,17919374230893380543,18392249325208771981
Rnd  10: 2135.86 ms → 14610129834974835703,16712037937771104037,18245068059073510541
Rnd  11:   13.67 ms → 12008150434885562717,13218389793977062229,17123373699922539397
Rnd  12: 3125.07 ms → 10114019147413009057,13697749200621089827,176